In [2]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import json
import numpy as np
import cv2
from matplotlib import pyplot

#python -m pip install -U scikit-image
#import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
#from skimage.io import imread

from skimage.io import imread
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import pickle

In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

dir = "C:/food-101/food-101/images/"
categories = os.listdir(dir)

randomSampleCount = 1000

random.seed(255)
sequence = [i for i in range(1000)]
randList = random.sample(sequence, randomSampleCount)

randomSampled = []
scaleddir = 'C:/food-101/food-101/imagesScaled/'
#for cat in categories:
for cat in categories[:5]:
    directory = scaleddir + cat
    path = os.listdir(directory)
    index = 0
    imagesToBeSampled = []
    for images in path:
        index = index + 1
        if(index in randList):
            imagesToBeSampled.append(images)
    randomSampled.append(imagesToBeSampled)
#print(randomSampled[0])

### CNN ###
from sklearn.preprocessing import MinMaxScaler

scaled_X = []
images2 = []

scaleddir = 'C:/food-101/food-101/imagesScaled/'

index = 0
for cat in categories[:5]:
    directory = scaleddir + cat    
    path = randomSampled[index]
    index = index + 1
    for images in path:
        toArr = imread(os.path.join(directory, images))
        img_resized = resize(toArr, (100, 100,3))
        #flat.append(toArr)
        scaled_X.append(img_resized)
        images2.append(categories.index(cat))

scaled_X = np.array(scaled_X)


ts = .30
rs = 45
x_train, x_test, y_train, y_test = train_test_split(
    scaled_X, 
    images2, 
    test_size=0.2, 
    random_state=rs, 
    stratify=images2)

In [21]:
x_train.shape

(3996, 100, 100, 3)

In [28]:
# (3,3) convolutional window, 2 pooling layers,  0.58 val acc
model_cnn = models.Sequential(
    [
        layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (100,100,3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64,(3,3), activation = 'relu'),
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(5),

        
    ])
model_cnn.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 8, 8, 64)        

In [29]:
x_test_tensor = tf.convert_to_tensor(x_test)
x_train_tensor = tf.convert_to_tensor(x_train)

y_test_tensor = tf.convert_to_tensor(y_test)
y_train_tensor = tf.convert_to_tensor(y_train)

model_cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model_cnn.fit(x_train_tensor, y_train_tensor, epochs=100, 
                    validation_data=(x_test_tensor, y_test_tensor))

Epoch 1/10
125/125 [==============================] - 20s 158ms/step - loss: 1.5421 - accuracy: 0.2795 - val_loss: 1.3727 - val_accuracy: 0.3884
Epoch 2/10
125/125 [==============================] - 14s 114ms/step - loss: 1.2733 - accuracy: 0.4552 - val_loss: 1.0893 - val_accuracy: 0.5556
Epoch 3/10
125/125 [==============================] - 14s 115ms/step - loss: 1.0954 - accuracy: 0.5420 - val_loss: 1.0751 - val_accuracy: 0.5566
Epoch 4/10
125/125 [==============================] - 15s 124ms/step - loss: 1.0148 - accuracy: 0.5693 - val_loss: 1.0040 - val_accuracy: 0.6006
Epoch 5/10
125/125 [==============================] - 16s 125ms/step - loss: 0.9481 - accuracy: 0.6074 - val_loss: 1.0575 - val_accuracy: 0.5846
Epoch 6/10
125/125 [==============================] - 16s 126ms/step - loss: 0.9157 - accuracy: 0.6256 - val_loss: 1.0219 - val_accuracy: 0.5886
Epoch 7/10
125/125 [==============================] - 16s 125ms/step - loss: 0.8507 - accuracy: 0.6474 - val_loss: 1.0253 - val_ac